In [1]:
import pandas as pd
import numpy as np


# 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import random

# 모델
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow import keras

# 시각화 툴
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
pd.options.mode.chained_assignment = None

1 Physical GPUs, 1 Logical GPUs


In [2]:
x_train_path = './content/train.csv'
y_train_path = './content/train_label.csv'
x_test_path = './content/test.csv'

x_train = pd.read_csv(x_train_path)
x_test = pd.read_csv(x_test_path)
y_train = pd.read_csv(y_train_path)

In [3]:
time_series_length = pd.concat([x_train['EMAIL'].value_counts()])
shortest_length = time_series_length[-1]

arranged_labels = []
for id in x_train['EMAIL'].unique():
    idx = x_train['EMAIL'][x_train['EMAIL'] == id].index
    start_idx = idx[0]
    end_idx = idx[-1]
    x_train.drop(list((range(start_idx + shortest_length, end_idx+1))), axis=0, inplace=True)
    x_train = x_train.reset_index(drop=True)
    label_idx = y_train['SAMPLE_EMAIL'][y_train['SAMPLE_EMAIL'] == id].index[0]
    arranged_labels.append(y_train['DIAG_NM'][label_idx])

In [4]:

for id in x_test['EMAIL'].unique():
    idx = x_test['EMAIL'][x_test['EMAIL'] == id].index
    start_idx = idx[0]
    end_idx = idx[-1]
    x_test.drop(list((range(start_idx + shortest_length, end_idx+1))), axis=0, inplace=True)
    x_test = x_test.reset_index(drop=True)

In [5]:
x_train.shape

(5180, 65)

In [6]:
#x_train = pd.read_csv(x_train_path)
#x_test = pd.read_csv(x_test_path)
x_train_org = x_train
x_test_org = x_test


# 불필요한 컬럼 제거 작업 'EMAIL',
drop = ['summary_date', 'EMAIL','activity_class_5min','activity_met_1min', 'sleep_hr_5min', 'sleep_hypnogram_5min',
        'sleep_is_longest', 'sleep_rmssd_5min', 'timezone', 'sleep_total','sleep_temperature_trend_deviation', 
        'CONVERT(activity_class_5min USING utf8)','CONVERT(activity_met_1min USING utf8)', 'CONVERT(sleep_hr_5min USING utf8)',
        'CONVERT(sleep_hypnogram_5min USING utf8)','CONVERT(sleep_rmssd_5min USING utf8)','activity_average_met', 'activity_cal_active', 'activity_cal_total','activity_steps', 'activity_score',
        'activity_score_move_every_hour', 'activity_daily_movement','activity_met_min_high',
        'activity_score_stay_active', 'activity_total', 'activity_met_min_low','activity_medium',
        'activity_score_training_frequency','activity_steps', 'activity_score',
        'activity_score_training_volume','sleep_efficiency','sleep_duration','sleep_hr_lowest',
        'sleep_midpoint_time','sleep_awake','sleep_deep','sleep_rem','sleep_restless','sleep_temperature_delta',
        'sleep_score_total','sleep_score','activity_score_meet_daily_targets','sleep_period_id','activity_inactivity_alerts',
        'activity_score_recovery_time','sleep_score_latency', 'sleep_score_alignment', 'sleep_score_efficiency']
x_train = x_train.drop(columns = drop)
test_email = x_test['EMAIL']
x_test = x_test.drop(columns = drop)


# month 추가
for i in range (0,len(x_train_org.iloc[:,1])):
  date = x_train_org['summary_date']
  temp = date[i].split('-')
  x_train.loc[i,'month'] = temp[1]
  
for i in range (0,len(x_test_org.iloc[:,1])):
  date = x_test_org['summary_date']
  temp = date[i].split('-')
  x_test.loc[i,'month'] = temp[1]

# 시계열 자료 수치화 보조 함수
def sumsquare_exp(llist):
  sum = 0
  #j = 2
  for i in llist:
    sum += np.exp(i)
  return sum
def sumsquare(llist):
  sum = 0
  j = 3
  for i in llist:
    sum = sum + i ** j # i 의 j제곱
  return sum
def interpolate(x):
    temp = x.split('/')[:-1]
    temp = pd.to_numeric(temp)
    temp = pd.Series(temp).astype('float').replace(0., np.NaN).interpolate().tolist()
    temp = [x for x in temp if np.isnan(x) == False]
    return temp 
## convert new features 추가
convert_low = ['CONVERT(activity_met_1min USING utf8)']
convert = ['CONVERT(sleep_hypnogram_5min USING utf8)']
for i in convert_low:
  x_train_org.loc[:,i] = x_train_org[i].apply(interpolate)
  x_test_org.loc[:,i] = x_test_org[i].apply(interpolate)
  for j in range (0,len(x_train_org.iloc[:,1])):
    x_train.loc[j, 'new_' + i] = sumsquare_exp(x_train_org.loc[j,i]) / len(x_train_org.loc[j,i])
  for k in range (0,len(x_test_org.iloc[:,1])):
    x_test.loc[k, 'new_' + i] = sumsquare_exp(x_test_org.loc[k,i]) / len(x_test_org.loc[k,i])
for i in convert:
  x_train_org.loc[:,i] = x_train_org[i].apply(interpolate)
  x_test_org.loc[:,i] = x_test_org[i].apply(interpolate)
  for j in range (0,len(x_train_org.iloc[:,1])):
    x_train.loc[j, 'new_' + i] = sumsquare(x_train_org.loc[j,i]) / len(x_train_org.loc[j,i])
  for k in range (0,len(x_test_org.iloc[:,1])):
    x_test.loc[k, 'new_' + i] = sumsquare(x_test_org.loc[k,i]) / len(x_test_org.loc[k,i])

In [7]:
x_test.shape

(16485, 20)

In [8]:
x_train.shape

(5180, 20)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)

In [10]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
arranged_labels = np.reshape(arranged_labels,(-1,1))
le = OneHotEncoder()
le.fit(arranged_labels)
arranged_labels = le.fit_transform(arranged_labels).toarray()
y = arranged_labels

In [25]:
X = x_train.reshape((148,35,20))

In [12]:
from sklearn.utils.class_weight import compute_class_weight

y_integers = np.argmax(y, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))

C:\Users\leePC\anaconda3\envs\tf2.4\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning:

Pass classes=[0 1 2], y=[0 2 2 0 2 0 2 0 2 0 0 2 1 0 0 0 0 2 2 0 2 0 0 0 2 1 0 1 0 2 0 2 2 0 0 2 0
 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 2 0 0 2 0 0 0 0 1 2 0 0 2 0 1 0 0 2 0 0
 0 0 1 0 0 2 2 0 0 0 2 0 2 0 2 0 0 1 0 0 0 0 2 2 2 0 2 0 0 0 0 0 1 2 2 2 0
 2 2 0 0 0 0 2 0 0 2 2 0 0 0 1 0 2 1 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error



In [26]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Input, Convolution2D, Activation,TimeDistributed
from keras.layers import Input, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import GlobalAveragePooling1D, Permute, Dropout
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

In [27]:

def generate_model():
    ip = Input(shape=(35, 20))

    x = Masking()(ip)
    x = LSTM(units = 50, return_sequences = True)(x)
    x = LSTM(units=50)(x)
    x = Dropout(0.8)(x)

  
    y = Reshape((700,1))(ip)
    y = Conv1D (kernel_size=3, filters=64, strides=3, padding='valid',kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    
    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)

    y = Conv1D (kernel_size=3, filters=64, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)

    
    y = Dropout(0.8)(y)
    y = Flatten()(y)
    
    x = concatenate([x, y])

    out = Dense(3, activation='softmax')(x)

    model = Model(ip, out)
    #model.summary()
    return model
model = generate_model()
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 35, 20)]     0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 700, 1)       0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 233, 64)      256         reshape_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 233, 64)      256         conv1d_3[0][0]                   
____________________________________________________________________________________________

In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4321)

In [29]:
from keras import optimizers
from kerastuner.tuners import BayesianOptimization, RandomSearch
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=10, mode='auto')

adams = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
early_stopping = EarlyStopping(patience=20, mode='auto')
#epoch 23 works best
model.compile(loss='categorical_crossentropy',
              optimizer=adams,
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test),class_weight=d_class_weights)

Epoch 1/300
4/4 [==============================] - 10s 983ms/step - loss: 3.3444 - accuracy: 0.4429 - val_loss: 1.1971 - val_accuracy: 0.6667
Epoch 2/300
4/4 [==============================] - 0s 34ms/step - loss: 3.0269 - accuracy: 0.3974 - val_loss: 1.1377 - val_accuracy: 0.6667
Epoch 3/300
4/4 [==============================] - 0s 36ms/step - loss: 4.2807 - accuracy: 0.3945 - val_loss: 1.1120 - val_accuracy: 0.6667
Epoch 4/300
4/4 [==============================] - 0s 35ms/step - loss: 3.4777 - accuracy: 0.4335 - val_loss: 1.1355 - val_accuracy: 0.6000
Epoch 5/300
4/4 [==============================] - 0s 33ms/step - loss: 2.5305 - accuracy: 0.4424 - val_loss: 1.2040 - val_accuracy: 0.5333
Epoch 6/300
4/4 [==============================] - 0s 34ms/step - loss: 2.8319 - accuracy: 0.3968 - val_loss: 1.2985 - val_accuracy: 0.5333
Epoch 7/300
4/4 [==============================] - 0s 34ms/step - loss: 2.6360 - accuracy: 0.3348 - val_loss: 1.3855 - val_accuracy: 0.4333
Epoch 8/300
4/4 [=

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_test)
x_test = scaler.transform(x_test)

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [31]:
x_test.shape

(471, 35, 20)

In [32]:
x_test = x_test.reshape((471, 35, 20))
yhat = model.predict(x_test)

In [33]:
test_labels = pd.read_csv('content/sample_submission.csv')
for i in range(0, 471):
    if yhat[i][0] > yhat[i][1] and yhat[i][0] > yhat[i][2]:
        test_labels['DIAG_NM'][i] = 'CN'
    elif yhat[i][1] > yhat[i][0] and yhat[i][1] > yhat[i][2]:
        test_labels['DIAG_NM'][i] = 'Dem'
    elif yhat[i][2] > yhat[i][1] and yhat[i][2] > yhat[i][0]:
        test_labels['DIAG_NM'][i] = 'MCI'

In [34]:
test_labels.to_csv('content/predict25.csv', index=False)